In [13]:
from keras.models import Sequential
from keras.layers import *
from keras.utils import np_utils
from imgUtils import *
from mailsUtils import send_mail
import time
import sys

### Define model

In [14]:
np.random.seed(123)  # for reproducibility
imgU = ImgUtils("./dataset", 10000)
start = time.strftime("%c")
theTrueScore = []
nb_classes = imgU.discover_and_make_order()

# Define model architecture
model = Sequential()

model.add(Convolution2D(nb_filter=64, nb_row=3, nb_col=3, activation='relu', input_shape=(256, 256, 3),
                        dim_ordering='tf'))
model.add(Convolution2D(nb_filter=32, nb_row=3, nb_col=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Discovering dataset...
2 classes found.
 1251 images found.
Shuffling order...
Ready for loading!
 938 for training and 312 for testing


/home/lux/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", input_shape=(256, 256,..., kernel_size=(3, 3), filters=64, data_format="channels_last")`
/home/lux/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", kernel_size=(3, 3), filters=32)`


In [15]:
redo = True
while redo:
    redo, x_train, y_train = imgU.load_dataset()
    # Preprocessing
    x_train = x_train.astype('float32')
    x_train /= 255

    y_train = np_utils.to_categorical(y_train, nb_classes)

    # Fit model on training data
    print("Starting...")
    model.fit(x_train, y_train, batch_size=10, epoch=10, verbose=0)
    # TODO: Maybe this is the wrong order of how to apply epochs -> investigate

print("Train completed! Will now evalutate...")

Loaded all imgs for training. Next call will load test data...
Loading completed!
Starting...


/home/lux/anaconda3/lib/python3.6/site-packages/keras/models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


KeyboardInterrupt: 

In [ ]:
# Evaluate
redo = True
while redo:
    redo, x_test, y_test = imgU.load_dataset()
    # Preprocessing
    x_test = x_test.astype('float32')
    x_test /= 255

    y_test = np_utils.to_categorical(y_test, nb_classes)
    # Evaluate model on test data
    theTrueScore.append(model.evaluate(x_test, y_test, batch_size=10, verbose=0))

In [ ]:
y_hat = model.predict_classes(x_test)
pd.crosstab(y_hat, y_test)

### Confusion matrix

In [ ]:
test_wrong = [im for im in zip(X_test,y_hat,y_test) if im[1] != im[2]]

plt.figure(figsize=(10, 10))
for ind, val in enumerate(test_wrong[:100]):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.subplot(10, 10, ind + 1)
    im = 1 - val[0].reshape((256,256))
    plt.axis("off")
    plt.text(0, 0, val[2], fontsize=14, color='blue')
    plt.text(8, 0, val[1], fontsize=14, color='red')
    plt.imshow(im, cmap='gray')